## 기본적인 모델 구성


간단하게, 제가 어떠한 모델을 구상하고, 이걸 적용하려고 하는지에 대한 설명이 필요할 것 같아서 이렇게 설명용 코드를 참고합니다. 필요한 부분들은 최대한 자세히 설명을 적어 놓겠습니다.

보고 조금 이해가 안되는 부분이 있다면, 그 부분은 저에게 직접 질문해주시면 감사하겠습니다.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Embedding,Dot,Add,Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD,Adamax
from tensorflow.keras.layers import Dense,Concatenate,Activation

모든 모델은 keras 기반으로 만들었습니다. (파이토치면 더 좋은데, 그걸 제가 어떻게 구현해야 할지 잘 모르겠네요)

In [ ]:
#성별과 연령에 관한 편향성이 존재한다고 생각
#위 코드와는 다르게 이번에는 점수를 학습시켜, 특정 성별,연령 에 따른 소비 점수를 구하고자 함
#(단, 소비의 경향성을 곱할 수도 있으나, 그런 부분은 bias에서 처리)
#사용한 소비 점수는 간단하게 (사용한금액)/(그종목에관한평균지출값) 으로 계산하여 과도한 금액으로 인한 편차를 지웠다.
#cold start와 적은 지표에 관해서는 매우 부정확한 결과를 가져올 수 있다는 단점이 있다 => 다만, 이건 모델 선에서 해결이 불가능하다.

#user = Input(shape=(1,1)) #=>만약, user를 특정할 수 있는 정보가 있다면, 이 input을 하나 더 사용하면 좋다.
hj = Input(shape=(1,))
sex = Input(shape=(1,))
age = Input(shape=(1,))
item_buz = Input(shape=(1,))

K = 200

#U_embedding = Embedding(2000,K,embeddings_regularizer=l2())(unit) #user를 특정 할 수 있는 정보가 있을 때 사용
H_embedding = Embedding(H,K,embeddings_regularizer=l2())(hj)
S_embedding = Embedding(S,K,embeddings_regularizer=l2())(sex)
A_embedding = Embedding(A,K,embeddings_regularizer=l2())(age)
I_embedding = Embedding(I,K,embeddings_regularizer=l2())(item_buz)

#user_bias = Embedding(U.max()+1,1,embeddings_regularizer=l2())(user) #user를 특정 할 수 있는 정보가 있을 때 사용
sex_bias = Embedding(S,1,embeddings_regularizer=l2())(sex)
age_bias = Embedding(A,1,embeddings_regularizer=l2())(age)
hj_bias = Embedding(H,1,embeddings_regularizer=l2())(hj)
item_bias = Embedding(I,1,embeddings_regularizer=l2())(item_buz)
from tensorflow.keras.layers import Dense,Concatenate,Activation

#U_embedding = Flatten()(U_embedding)
H_embedding = Flatten()(H_embedding)
S_embedding = Flatten()(S_embedding)
A_embedding = Flatten()(A_embedding)
I_embedding = Flatten()(I_embedding)

#user_bias = Flatten()(user_bias)
sex_bias = Flatten()(sex_bias)
age_bias = Flatten()(age_bias)
hj_bias = Flatten()(hj_bias)
item_bias = Flatten()(item_bias)

#user_bias 와 U_embedding을 user를 특정 할 수 있는 정보가 있다면 추가하면 좋다. => 더 높은 정확성과, 적은 오류를 가질 것이다.
R = Concatenate()([S_embedding,A_embedding,H_embedding,I_embedding,sex_bias,age_bias,hj_bias,item_bias])

R = Dense(2048)(R)
R = Activation('relu')(R)
R = Dense(256)(R)
R = Activation('relu')(R)
R = Dense(1)(R) #소비 점수를 output으로 가져온다(정확한 평가 지표가 있었다면, 조금 더 좋았을 것)

model = Model(inputs=[sex,age,hj,item_buz],outputs=R)
model.compile(
    loss = RMSE,
    optimizer = 'Adam',
    metrics = [RMSE]
)

model.summary()

<img src='결과물/pic/layer.png'>

모델 자체에는 어려운 내용이 없습니다.

우선 특이한 점 하나는, '모든 사용자 ID' '모든 아이템 ID'를 그냥 일반 categorical labeling을 해버린다는 점입니다.

기본적으로, 머신러닝에서는 이러한 일괄적인 categorical labeling이 좋지 않습니다. 왜냐면, 1번 사용자와 2번 사용자는 사실상 차이가 존재하지 않는데, 이걸 그냥 레이블링을 해버린다면

2>1로 인해 학습이 ID가 큰 쪽에 어떤 영향을 받도록 모델링이 될수도 있습니다. 그래서 One hot encoding을 많이들 사용하실텐데, 그걸 수백 수천개의 ID에 사용하는 것은

말도 안되는 일입니다.(Feature의 수를 천개 이상 늘린다는 말도 안되는 상황이니.. => PCA를 하자! 라고 할수도있는데, 그럼 원본  Feature들에 대한 정보가 전부 손실되어 버립니다.)

그래서 여기서 우리는 **Embedding Layer** 라는 친구를 사용합니다. 이 친구는 자동적으로 들어온 전체 아이디수를 바탕으로(예를들어 N명의 사용자가 있다고 해봅시다) N X N 행렬을 만듭니다.

그리고, 만약 2번 사용자가 들어온다면, 2행 2열의 부분만 1로 설정하고 나머지는 전부 0으로 바꾼 행렬을 생성하여 **자동적으로 ID를 행렬을 통해 구분** 합니다.

이건 이제 마찬가지로 Movie 항목에도 똑같이 적용됩니다. 자, 그럼 이제 이 큰 행렬을 그냥 집어 넣으면 될까요?

아쉽게도 그건 너무나도 비효율적입니다. 그래서 우리는이 Embedding Layer를 Flatten() 이라는 모듈에 집어넣어, 컴퓨터가 이해할 수 있는 벡터의 형태로 변환시켜 줍니다.

<img src='결과물/pic/flatten.png'>

flatten을 거친다면, 우리가 만든 행렬은 자동적으로 학습할 수 있는 상태로 변합니다. 그래서 마지막으로 이 벡터로된 입력들을 하나로 합칩니다.

이 과정이 Concatenate() 모듈입니다.

이렇게 우리는 **이러한 입력을 받을것이다!** 라고 설정한것이 위 코드의 **R** 이라는 부분이고, 이제 이 아래부터 딥러닝을 위한 신경망을 쌓으면 됩니다

몇가지 우리의 모델이 특이한 점이 있습니다.

1. 우리의 모델은 OOO_bias 라는 무언가 특이한 부분이 있습니다. 이 bias는 쉽게 이야기하자면 '특정 input에 대한 경향성을 다시 input으로 만든것' 입니다.
예를들어 성별에 따른 특정한 경향성이 있다고 말해봅시다. 여성의 경우 의류점을, 남성의 경우 취미소매점을 갈 확률이 높다고 데이터에서 확인이 된다면, 이러한 경향성또한  input으로 넣을 수 있습니다. 이를 통해 성능의 개선을 이끌수 있습니다

2. Embedding 라인에 보시면 H,S,A,I 와 같은 알수없는 알파벳이 들어가있는데, 이것들은 '특정 입력에 대한 그 입력의 수' 를 나타내는 알파벳입니다. 예를들어 사용자가 총 200명이라면, 그 사용자의 Embedding layer에서의 알파벳은 200을 나타내는 변수일 것입니다.

3. RMSE를 사용해서 loss를 계산합니다. 이건 어찌보면 당연한데, 우리의 모델은 '추측'을 기반으로 하기에, RMSE이가 사용하기에 매우 적합합니다. (Accuracy는 classification 이 아님으로 불가능합니다.)